In [145]:
import pandas as pd
import numpy as np

np.random.seed(42)  # for reproducibility
n = 20
companies = [f"Company_{i}" for i in range(1, n+1)]
sectors = ["Energy", "Finance", "Tech", "Healthcare", "Consumer"]
ratings = ["AAA", "AA", "A", "BBB", "BB", "B"]
rating_weights = [0.1, 0.15, 0.25, 0.25, 0.15, 0.1]  # to reflect reality

portfolio = pd.DataFrame({
    "Company": companies,
    "Sector": np.random.choice(sectors, size=n),
    "Rating": np.random.choice(ratings, size=n, p=rating_weights),
    "CouponRate": np.random.uniform(5, 12, size=n),
    "MaturityYears": np.random.randint(1, 11, size=n),
    "MarketPrice": np.random.uniform(90, 110, size=n),
    "FaceValue": 100
})


In [146]:
portfolio.head()

,Company,Sector,Rating,CouponRate,MaturityYears,MarketPrice,FaceValue
0,Company_1,Healthcare,BBB,5.111764,7,104.226839,100
1,Company_2,Consumer,AAA,6.616257,9,105.803511,100
2,Company_3,Tech,AAA,6.687178,8,102.119199,100
3,Company_4,Consumer,BBB,9.782845,5,108.526018,100
4,Company_5,Consumer,A,9.269977,2,103.021541,100


In [147]:
portfolio["YTM"] = (portfolio["CouponRate"] / portfolio["MarketPrice"]) * 100
portfolio[["Company", "CouponRate", "MarketPrice", "YTM"]].head()


,Company,CouponRate,MarketPrice,YTM
0,Company_1,5.111764,104.226839,4.904460
1,Company_2,6.616257,105.803511,6.253343
2,Company_3,6.687178,102.119199,6.548405
3,Company_4,9.782845,108.526018,9.014285
4,Company_5,9.269977,103.021541,8.998096


In [148]:
portfolio["Duration"] = portfolio["MaturityYears"] / (1 + (portfolio["YTM"] / 100))
portfolio["Common Size"] = portfolio["Duration"]/ portfolio["MaturityYears"]
portfolio[["Company", "MaturityYears", "YTM", "Duration","Common Size"]].head()

,Company,MaturityYears,YTM,Duration,Common Size
0,Company_1,7,4.904460,6.672738,0.953248
1,Company_2,9,6.253343,8.470322,0.941147
2,Company_3,8,6.548405,7.508325,0.938541
3,Company_4,5,9.014285,4.586555,0.917311
4,Company_5,2,8.998096,1.834894,0.917447


In [149]:
rating_map = {"AAA": 1, "AA": 2, "A": 3, "BBB": 4, "BB": 5, "B": 6}
portfolio["RatingNum"] = portfolio["Rating"].map(rating_map)

portfolio_metrics = {
    "Average_YTM": portfolio["YTM"].mean(),
    "Weighted_Avg_Rating": np.average(portfolio["RatingNum"], weights=portfolio["FaceValue"]),
    "Average_Duration": portfolio["Duration"].mean(),
    "Sector_Concentration": portfolio["Sector"].value_counts(normalize=True).to_dict()
}



In [150]:
portfolio_metrics

{'Average_YTM': 7.963555107292021,
 'Weighted_Avg_Rating': 3.25,
 'Average_Duration': 5.894449277680311,
 'Sector_Concentration': {'Consumer': 0.3,
  'Healthcare': 0.25,
  'Tech': 0.25,
  'Finance': 0.15,
  'Energy': 0.05}}

In [151]:
import plotly.express as px
sector_counts = portfolio["Sector"].value_counts()

fig = px.pie(portfolio, values= sector_counts.values, names = sector_counts.index)
fig.show()


In [152]:
fig2 = px.histogram(
    portfolio,
    x="Duration",
    nbins=10,
    title="Distribution of Bond Durations"
)
fig2.show()


In [153]:
fig2 = px.histogram(
    portfolio,
    x="Duration",
    nbins=20,
    title="Distribution of Bond Durations"
)
fig2.show()


In [154]:
fig3 = px.scatter(
    portfolio,
    x="Rating",
    y="YTM",
    color="Sector",  # optional: color by Sector
    hover_data=["Company"],
    title="YTM vs Credit Rating",
    category_orders={
        "Rating": ["AAA", "AA", "A", "BBB", "BB", "B"]
    }

)
fig3.show()


In [155]:
# ----------------------------
# LEVEL 2: Interest Rate Shock Simulation
# ----------------------------
rate_shock = 1.00  # 1% interest rate hike

# Modified Duration calculation
portfolio["ModDuration"] = portfolio["Duration"] / (1 + (portfolio["YTM"] / 100))


In [156]:
# Estimate price drop due to rate hike
portfolio["EstimatedPriceAfterShock"] = portfolio["MarketPrice"] * (1 - (portfolio["ModDuration"] * rate_shock / 100))
portfolio["Loss"] = portfolio["MarketPrice"] - portfolio["EstimatedPriceAfterShock"]
portfolio["LossPercent"] = (portfolio["Loss"] / portfolio["MarketPrice"]) * 100

In [157]:
# Total Portfolio Loss
total_portfolio_loss = portfolio["Loss"].sum()


In [158]:
print("Total Portfolio:", portfolio["MarketPrice"].sum())
print("Total Loss:", total_portfolio_loss)


Total Portfolio: 2045.7765303877206
Total Loss: 111.95955727562183


In [159]:
# Sector-wise Loss
sector_loss = portfolio.groupby("Sector")["Loss"].sum().reset_index()
sector_loss = sector_loss.sort_values(by="Loss", ascending=False)

In [160]:
sector_exposure = portfolio.groupby("Sector")["MarketPrice"].sum().reset_index()
sector_exposure

,Sector,MarketPrice
0,Consumer,623.571666
1,Energy,104.434590
2,Finance,301.452167
3,Healthcare,512.982473
4,Tech,503.335634


In [161]:
sector_loss

,Sector,Loss
4,Tech,38.843342
0,Consumer,34.906388
3,Healthcare,22.570812
2,Finance,13.033761
1,Energy,2.605254


In [162]:

# VISUALIZATION: Loss by Sector
# ----------------------------
fig_loss_sector = px.bar(
    sector_loss,
    x="Sector",
    y="Loss",
    title="Estimated Sector-Wise Loss After 1% Rate Hike",
    text="Loss",
    color="Sector",
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig_loss_sector.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig_loss_sector.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig_loss_sector.show()


In [163]:

# ----------------------------
# SUMMARY OUTPUT
# ----------------------------
print("\n--- Total Portfolio Loss After 1% Rate Shock ---")
print(f"₹{total_portfolio_loss:,.2f}")

print("\n--- Portfolio With Shock Impact ---")
print(portfolio[["Company", "Sector", "Rating", "YTM", "Duration", "ModDuration", "MarketPrice", "EstimatedPriceAfterShock", "Loss", "LossPercent"]].sort_values(by="LossPercent"))



--- Total Portfolio Loss After 1% Rate Shock ---
₹111.96

--- Portfolio With Shock Impact ---
       Company      Sector Rating        YTM  Duration  ModDuration  \
15  Company_16     Finance      A   8.343642  0.922989     0.851909   
10  Company_11  Healthcare      A   7.715550  0.928371     0.861873   
19  Company_20  Healthcare      A   7.691943  0.928575     0.862251   
4    Company_5    Consumer      A   8.998096  1.834894     1.683419   
18  Company_19      Energy    BBB   9.662415  2.735668     2.494627   
5    Company_6     Finance    AAA  10.002258  4.545361     4.132062   
3    Company_4    Consumer    BBB   9.014285  4.586555     4.207297   
12  Company_13    Consumer      A   8.812981  6.433056     5.912030   
0    Company_1  Healthcare    BBB   4.904460  6.672738     6.360776   
14  Company_15  Healthcare    BBB  10.764147  7.222554     6.520660   
17  Company_18    Consumer      B  10.737331  7.224303     6.523819   
16  Company_17  Healthcare    AAA   7.096546  7.46989

#multi-scenario rate shock simulation

In [164]:
shock_levels = [0.5, 1.0, 1.5, 2.0]
shock_results = []

for shock in shock_levels:
    est_price = portfolio["MarketPrice"] * (1 - (portfolio["ModDuration"] * shock / 100))
    total_loss = (portfolio["MarketPrice"] - est_price).sum()
    
    shock_results.append({
        "Shock_%": shock,
        "Total_Loss": total_loss
    })


In [165]:
shock_df = pd.DataFrame(shock_results)
print(shock_df)


   Shock_%  Total_Loss
0      0.5   55.979779
1      1.0  111.959557
2      1.5  167.939336
3      2.0  223.919115


In [166]:
import plotly.express as px

fig = px.line(
    shock_df,
    x="Shock_%",
    y="Total_Loss",
    markers=True,
    title="Portfolio Loss vs Interest Rate Shock",
    labels={"Shock_%": "Rate Shock (%)", "Total_Loss": "Total Loss (₹)"}
)

fig.update_traces(
    mode="lines+markers",
    line=dict(color="crimson", width=3)
)

fig.update_layout(
    yaxis_tickprefix="₹",
    xaxis=dict(dtick=0.5)
)

fig.show()


#Credit Downgrade Stress Test

In [167]:
# Define ordered rating scale
rating_order = ["AAA", "AA", "A", "BBB", "BB", "B"]

# Clone original portfolio
portfolio_downgraded = portfolio.copy()

# Pick 20% (4 bonds) to downgrade
downgrade_indices = np.random.choice(portfolio.index, size=4, replace=False)

# Downgrade function
def downgrade_rating(r):
    idx = rating_order.index(r)
    return rating_order[min(idx + 1, len(rating_order) - 1)]  # max downgrade = B

# Apply downgrade
portfolio_downgraded.loc[downgrade_indices, "Rating"] = portfolio_downgraded.loc[downgrade_indices, "Rating"].apply(downgrade_rating)


In [168]:
# Count pre- and post-downgrade rating distribution
before_counts = portfolio["Rating"].value_counts().sort_index(key=lambda x: [rating_order.index(i) for i in x])
after_counts = portfolio_downgraded["Rating"].value_counts().reindex(before_counts.index, fill_value=0)

# Combine into one DataFrame
rating_comparison = pd.DataFrame({
    "Rating": before_counts.index,
    "Before": before_counts.values,
    "After": after_counts.values
})


In [169]:
rating_comparison

,Rating,Before,After
0,AAA,5,4
1,AA,1,2
2,A,5,2
3,BBB,5,8
4,BB,1,1
5,B,3,3


In [170]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Bar(
    x=rating_comparison["Rating"],
    y=rating_comparison["Before"],
    name="Before Downgrade",
    marker_color="steelblue"
))

fig.add_trace(go.Bar(
    x=rating_comparison["Rating"],
    y=rating_comparison["After"],
    name="After Downgrade",
    marker_color="indianred"
))

fig.update_layout(
    barmode='group',
    title="Credit Rating Distribution Before vs After Downgrade",
    xaxis_title="Credit Rating",
    yaxis_title="Number of Bonds",
    bargap=0.2
)

fig.show()


What a Risk Manager Would Say:
“In our simulated downgrade, the portfolio’s high-grade exposure (AAA, A) eroded slightly, while BBB exposure rose. However, sub-investment-grade exposure remains stable, suggesting the portfolio’s credit risk profile has weakened modestly, but not critically.”



In [171]:
# Map new ratings to numbers
portfolio_downgraded["RatingNum"] = portfolio_downgraded["Rating"].map(rating_map)

# Calculate new weighted average rating
new_weighted_avg_rating = np.average(
    portfolio_downgraded["RatingNum"],
    weights=portfolio_downgraded["FaceValue"]
)

# Compare to original
original_rating = np.average(
    portfolio["RatingNum"],
    weights=portfolio["FaceValue"]
)

print(f"Original Weighted Avg Rating: {original_rating:.2f}")
print(f"New Weighted Avg Rating     : {new_weighted_avg_rating:.2f}")


Original Weighted Avg Rating: 3.25
New Weighted Avg Rating     : 3.45


In [172]:
# Create downgrade flag
downgraded_flags = portfolio["Rating"] != portfolio_downgraded["Rating"]
portfolio_downgraded["WasDowngraded"] = downgraded_flags

# View downgraded companies
downgraded_companies = portfolio_downgraded[portfolio_downgraded["WasDowngraded"]][
    ["Company", "Sector", "Rating", "WasDowngraded"]
]

print("🔻 Downgraded Bonds:")
print(downgraded_companies)


🔻 Downgraded Bonds:
       Company      Sector Rating  WasDowngraded
1    Company_2    Consumer     AA           True
10  Company_11  Healthcare    BBB           True
12  Company_13    Consumer    BBB           True
19  Company_20  Healthcare    BBB           True


In [173]:
print("📘 Final Portfolio Stress Summary")
print("---------------------------------")
print(f"➡ Average YTM             : {portfolio['YTM'].mean():.2f}%")
print(f"➡ Average Duration        : {portfolio['Duration'].mean():.2f} years")
print(f"➡ Total Loss @ 1% Shock   : ₹{total_portfolio_loss:,.2f}")
print(f"➡ Pre-Downgrade Rating    : {original_rating:.2f} (~A-)")
print(f"➡ Post-Downgrade Rating   : {new_weighted_avg_rating:.2f} (~BBB+)")
print(f"➡ Downgraded Bonds Count  : {downgraded_companies.shape[0]}")


📘 Final Portfolio Stress Summary
---------------------------------
➡ Average YTM             : 7.96%
➡ Average Duration        : 5.89 years
➡ Total Loss @ 1% Shock   : ₹111.96
➡ Pre-Downgrade Rating    : 3.25 (~A-)
➡ Post-Downgrade Rating   : 3.45 (~BBB+)
➡ Downgraded Bonds Count  : 4


#
📄 Credit Portfolio Risk Assessment – Stress Simulation Summary
Analyst: Akshay
Firm: Simulated Apollo Global Management Internship
Date: [insert today's date]

🧾 Portfolio Overview:
The portfolio consists of 20 synthetic corporate bonds diversified across 5 sectors with varying credit ratings from AAA to B. The average YTM is 7.96%, with an average Macaulay Duration of 5.89 years, indicating moderate interest rate sensitivity.

Metric	Value
Average YTM	7.96%
Average Duration	5.89 years
Weighted Avg Rating	3.25 (~A-)
Largest Sector	Consumer (30%)

🔺 Rate Shock Scenario: +1% Interest Rate
Using Modified Duration, a +1% parallel shift in interest rates was applied.

Impact Metric	Value
Total Estimated Loss	₹X (calc from code)
Max Individual Bond Loss %	~8.6%
Most Sensitive Bonds	Duration > 8 yrs
Sector Most Affected	[Sector] (from plot)

📌 Insight: Bonds with higher duration exhibited 7–8% price drops. Despite some yielding 8%+, duration-driven loss dominates short-term impact. Portfolio loss scaled linearly with shocks due to absence of convexity modeling.

🔻 Credit Downgrade Scenario: 20% Bonds Downgraded by 1 Notch
Simulated downgrade of 4 random bonds by one credit notch.

Rating Metric	Before	After
Weighted Avg Rating	3.25 (~A-)	3.45 (~BBB+)
Downgraded Bonds Count	4	
Increase in BBB Exposure	+3	
Sub-Investment Grade Bonds	No increase observed	

📌 Insight: Mild erosion in credit quality. The downgrade did not breach investment-grade thresholds. Exposure shifted toward BBB, increasing mid-grade credit risk without introducing junk-level volatility.

🔍 Commentary & Analyst View:
“The portfolio’s current positioning reflects a yield-seeking tilt with moderate interest rate exposure. The +1% rate shock wiped out ~6–8% of portfolio value for longer-duration names, flagging the need for possible hedging or asset reallocation if rates continue trending higher.

The downgrade simulation, while modest, highlights concentration risk in the BBB corridor. In a real-world context, a sector-specific downgrade wave (e.g., in Consumer or Healthcare) could exacerbate credit risk further.

Further analysis is recommended to overlay macro triggers (like inflation, tightening cycles) and their impact on rating migration probability. Consider developing a predictive downgrade risk model in Level 3 using financial ratios.”